In [1]:
import sys

sys.path.append("../")
from time_difference_matrix import Time_Difference_Matrix
from difference_matrix import Difference_Matrix

import numpy as np

### General Performance tests for matrix inversion algorithms

Let's create a second order difference matrix

In [2]:
D=Difference_Matrix(100,1,style="lapack")

In [3]:
D.D

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]])

In [4]:
D.compute_k_difference(0)

array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [5]:
D.D.shape

(98, 100)

### Confirm DDT inverse computed correctly via LAPACK

In [6]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

In [7]:
D=Difference_Matrix(100,2,style="sparse")

In [8]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(100-3),rtol=1e-8,atol=1e-8))

### Large computations of Difference Matrix


In [9]:
D=Difference_Matrix(1000,1,style="lapack")

In [10]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(1000-2),rtol=1e-4,atol=1e-4))

In [11]:
D=Difference_Matrix(1000,1,style="sparse")


In [12]:
assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(1000-2),rtol=1e-4,atol=1e-4))

### High order difference matrix

In [13]:
D=Difference_Matrix(100,4,style="lapack")
D.D

array([[  1.,  -5.,  10., ...,   0.,   0.,   0.],
       [  0.,   1.,  -5., ...,   0.,   0.,   0.],
       [  0.,   0.,   1., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  -1.,   0.,   0.],
       [  0.,   0.,   0., ...,   5.,  -1.,   0.],
       [  0.,   0.,   0., ..., -10.,   5.,  -1.]])

In [14]:
(D.DDT.dot(D.DDT_inv)-np.eye(100-5)).max()


3.8743019104003906e-05

### Confirm Difference Matrix Recursion works

In [15]:
D=Difference_Matrix(2500,1,style="lapack")
d_21=D.compute_k_difference(21)

In [16]:
D=Difference_Matrix(2500,21,style="sparse")


In [17]:
assert (np.allclose(d_21,D.D))

### Time Difference Matrix Computation

In [18]:
D=Difference_Matrix(100,1,style="lapack")

T=Time_Difference_Matrix(D,t=np.linspace(0,1,100))

In [19]:
assert(np.allclose(T.DDT.dot(T.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

In [20]:
D=Difference_Matrix(100,1,style="sparse")

T=Time_Difference_Matrix(D,t=np.linspace(0,1,100))

In [21]:
assert(np.allclose(T.DDT.dot(T.DDT_inv),np.eye(100-2),rtol=1e-8,atol=1e-8))

### Edge Cases

In [22]:
D=Difference_Matrix(100,0)

assert(np.allclose(D.DDT.dot(D.DDT_inv),np.eye(100-1),rtol=1e-8,atol=1e-8))

In [23]:
T=Time_Difference_Matrix(D,t=np.linspace(0,1,101))

In [24]:
D=Difference_Matrix(100,0)
D.D

array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [25]:
truncated_D=Difference_Matrix(99,0)
truncated_D.D


array([[-1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0., -1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0., -1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  1.]])

In [26]:
truncated_D.D.dot(D.D)

array([[ 1., -2.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1., -2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ..., -2.,  1.,  0.],
       [ 0.,  0.,  0., ...,  1., -2.,  1.]])